In [1]:
from gensim import corpora, models
import gensim
import nltk
nltk.download('stopwords')
nltk.download('wordnet')  
nltk.download('omw-1.4')  
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string
import json 
import pandas as pd

c:\Users\jmanu\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmanu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jmanu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jmanu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
import pandas as pd
df = pd.read_csv("../Datasets/PubMed/PubMed_20k_RCT/test.csv")

In [82]:
# # Sentences as Doc
# abstracts = df.abstract_text.values.tolist()

# # Abstract as Doc
abstracts = df.groupby('abstract_id')['abstract_text'].apply(' '.join)
articles = abstracts.tolist()

In [2]:
with open("./articles.json", 'r') as file:
    data = json.load(file)

In [108]:
# Abstract As documents
articles = [data[i]["Abstract"] for i in range(len(data))]


# All sentences of Abstract as Document
#articles = [sentence for i in range(len(data)) for sentence in data[i]["Abstract"].split("\n")]


In [122]:
# remove stopwords, punctuation, and normalize the corpus
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

clean_corpus = [clean(doc).split() for doc in articles]

# Step 2: Create Dictionary and Corpus
# Creating document-term matrix 
dictionary = corpora.Dictionary(clean_corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_corpus]

# Step 3: Build LDA model
from gensim.models import LsiModel
lda_model = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix, id2word=dictionary, random_state=42,num_topics=5, update_every=1, chunksize=100, passes=10)
#lsa = LsiModel(doc_term_matrix, num_topics=3, id2word = dictionary)


In [123]:
# View the topics to understand what each topic represents
for idx, topic in lda_model.print_topics(-1, num_words=8):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"digital" + 0.023*"twin" + 0.021*"process" + 0.013*"medical" + 0.011*"model" + 0.011*"used" + 0.010*"physical" + 0.010*"data"
Topic: 1 
Words: 0.017*"student" + 0.015*"ecases" + 0.013*"ecg" + 0.012*"clinical" + 0.011*"classification" + 0.011*"soo" + 0.011*"otva" + 0.010*"machine"
Topic: 2 
Words: 0.036*"patient" + 0.026*"pembrolizumab" + 0.020*"progression" + 0.020*"pd" + 0.016*"salvage" + 0.015*"study" + 0.014*"trial" + 0.014*"beyond"
Topic: 3 
Words: 0.013*"model" + 0.012*"data" + 0.012*"health" + 0.009*"digital" + 0.008*"system" + 0.008*"technology" + 0.007*"twin" + 0.007*"care"
Topic: 4 
Words: 0.038*"stage" + 0.038*"reversal" + 0.022*"patient" + 0.020*"diabetes" + 0.020*"study" + 0.020*"therapy" + 0.018*"2" + 0.015*"nutrition"


In [133]:
topic_labels = {
    0: 'Digital Twin',
    1: 'Clinical Trail',
    2: 'Patient Study',
    3: 'AI in Health',
    4: 'Medical Recovery'
}

In [132]:
print(articles[3])

Abstract All people are unique and so are their diseases.
Our genomes, disease histories, behavior, and lifestyles are all different; therefore it is not too surprising that people often respond differently when administered the same drugs.
Cancer, in particular, is a complex and heterogeneous disease, originating in patients with different genomes, in cells with the different epigenomes, formed and evolving on the basis of random processes, with the response to therapy not only depending on the individual cancer cell but also on many features of the patient.
Selection of an optimal therapy will therefore require a deep molecular analysis comprising both the patient and their tumor (e.g., comprehensive molecular tumor analysis [CMTA]), and much better personalized prediction of response to possible therapies.            As the knowledge base on cancer, cellular transduction and molecular interactions widens, so does our ability to generate computational models with the capacity to accu

In [134]:
results = []
# Example of mapping a document to a topic label
for i, row_list in enumerate(lda_model[doc_term_matrix]):
    row = row_list[0] if lda_model.per_word_topics else row_list            
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    dominant_topic = int(row[0][0])
    label = topic_labels[dominant_topic]
    results.append(label)
    print(f"Document {i} is labeled as: {label}")

Document 0 is labeled as: Digital Twin
Document 1 is labeled as: AI in Health
Document 2 is labeled as: Digital Twin
Document 3 is labeled as: Patient Study
Document 4 is labeled as: AI in Health
Document 5 is labeled as: Patient Study
Document 6 is labeled as: AI in Health
Document 7 is labeled as: AI in Health
Document 8 is labeled as: AI in Health
Document 9 is labeled as: AI in Health
Document 10 is labeled as: AI in Health
Document 11 is labeled as: Digital Twin
Document 12 is labeled as: AI in Health
Document 13 is labeled as: AI in Health
Document 14 is labeled as: Patient Study
Document 15 is labeled as: Patient Study
Document 16 is labeled as: Clinical Trail
Document 17 is labeled as: AI in Health
Document 18 is labeled as: Patient Study
Document 19 is labeled as: AI in Health
Document 20 is labeled as: Digital Twin
Document 21 is labeled as: Digital Twin
Document 22 is labeled as: AI in Health
Document 23 is labeled as: Clinical Trail
Document 24 is labeled as: Medical Recove

In [148]:
df_results = pd.DataFrame(results)
save_test(results)

In [147]:
def save_test(answers):
    file_path = "../Test_Results/Test Results.xlsx"
    sheet_name = "class_lab"
    answers.insert(0, "Topic Modeling")
    new_data = pd.DataFrame([answers])

    # Try to read the existing data
    try:
        existing_data = pd.read_excel(file_path, sheet_name=sheet_name)
    except FileNotFoundError:
        existing_data = pd.DataFrame()

    # Append new data
    updated_data = pd.concat([existing_data, new_data], ignore_index=True)
    updated_data = updated_data.drop_duplicates()

    # Write back to Excel
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        updated_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [12]:

# Assuming lda_model and corpus are already created as shown in the previous example

# Function to get the dominant topic for each document
def format_topics_sentences(ldamodel, corpus):
    # Init output
    topics_data = []
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the dominant topic
        dominant_topic = int(row[0][0])
        topics_data.append(dominant_topic)
        
    return topics_data


In [13]:
# Get dominant topic for each document
dominant_topics = format_topics_sentences(lda_model, clean_corpus)



ValueError: too many values to unpack (expected 2)

In [38]:
dominant_topics

[1,
 1,
 1,
 1,
 1,
 4,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 4,
 4,
 1,
 3,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 4,
 4,
 2,
 1,
 2,
 4,
 4,
 4,
 3,
 3,
 3,
 1,
 3,
 2,
 1,
 2,
 3,
 2,
 4,
 1,
 0,
 0,
 1,
 0,
 0,
 3,
 1,
 1,
 1,
 0,
 2,
 3,
 4,
 3,
 0,
 1,
 0,
 0,
 2,
 1,
 3,
 2,
 0,
 2,
 4,
 0,
 4,
 4,
 0,
 1,
 3,
 4,
 1,
 4,
 4,
 1,
 2,
 1,
 2,
 2,
 1,
 4,
 1,
 4,
 0,
 3,
 1,
 3,
 0,
 3,
 0,
 2,
 2,
 0,
 2,
 2,
 3,
 4,
 2,
 3,
 2,
 4,
 0,
 0,
 0,
 3,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 4,
 4,
 2,
 1,
 2,
 4,
 2,
 4,
 2,
 0,
 0,
 2,
 0,
 4,
 3,
 4,
 2,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 2,
 0,
 0,
 4,
 4,
 1,
 4,
 3,
 3,
 0,
 4,
 4,
 4,
 2,
 2,
 3,
 4,
 0,
 4,
 0,
 0,
 4,
 1,
 4,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 3,
 2,
 3,
 2,
 2,
 0,
 1,
 0,
 3,
 2,
 0,
 3,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 4,
 1,
 4,
 1,
 0,
 1,
 1,
 4,
 1,
 1,
 1,
 3,
 3,
 3,
 1,
 4,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 3,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 3,
 1,
 3,
 1,
 1,
 3,
 0,
 2,
